In [2]:
import requests
# 用来检测一个字符串的编码
import cchardet
# 看捕获的异常是什么内容
import traceback

In [11]:
def downloader(url, timeout=10, headers=None, debug=False, binary=False):
    _headers = {
        'User-Agent':('Mozilla/5.0 (compatible; MSIE 9.0; ''Windows NT 6.1; Win64; x64; Trident/5.0)')
    }
    redirected_url = url;
    if headers:
        _headers = headers
    try:
        r = requests.get(url, headers=_headers, timeout=timeout)
        if binary:
            html = r.content
        else:
            encoding = cchardet.detect(r.content)['encoding']
            html = r.content.decode(encoding)
        status = r.status_code
        redirected_url = r.url
    except:
        if debug:
            traceback.print_exc()
        msg = 'failed download:{}'.fotmat(url)
        print(msg)
        if binary:
            html = b''
        else:
            html = ''
        status = 0
    return status, html, redirected_url

if __name__ == '__main__':
    url = 'http://news.baidu.com/'
    s, html, lost_url_found_by_jjj = downloader(url)
    print(s, len(html), lost_url_found_by_jjj)

200 71197 http://news.baidu.com/


In [8]:
h = b''
print(h)
print(type(h))

b''
<class 'bytes'>


### URL的清洗
- http://xinwen.eastday.com/a/n181106070849091.html?qid=news.baidu.com
- http://news.ifeng.com/a/20181106/60146589_0.shtml?_zbs_baidu_news
- 上面两个带?的网站来自百度新闻的首页，这个问号?的作用就是告诉目标服务器，这个网址是从百度新闻链接过来的，是百度带过来的流量。但是它们的表示方式不完全一样，一个是qid=news.baidu.com， 一个是_zbs_baidu_news。这有可能是目标服务器要求的格式不同导致的，这个在目标服务器的后台的浏览统计程序中可能用得到。
然后去掉问号?及其后面的字符，发现它们和不去掉指向的是相同的新闻网页。
从字符串对比上看，有问号和没问号是两个不同的网址，但是它们又指向完全相同的新闻网页，说明问号后面的参数对响应内容没有任何影响。
正在抓取新闻的大量实践后，我们发现了这样的规律：
新闻类网址都做了大量SEO，它们把新闻网址都静态化了，基本上都是以.html, .htm, .shtml等结尾，后面再加任何请求参数都无济于事。
但是，还是会有些新闻网站以参数id的形式动态获取新闻网页。
那么我们抓取新闻时，就要利用这个规律，防止重复抓取。由此，我们实现一个清洗网址的函数。

### 切记，不要相信requests返回的encoding，自己判断一下更放心